Process Attraction Features Data
This script reads the attraction features data from JSON and converts it to a parquet file

import pandas as pd
import json
from pathlib import Path

In [9]:
import pandas as pd
import json
from pathlib import Path


Define paths

In [18]:
data_dir = Path('../data')
json_file = data_dir / 'attraction_features.json'
parquet_file = data_dir / 'attraction_features.parquet'


Read JSON data

In [19]:
# Read JSON data
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Expand the dictionary into separate columns
df_expanded = pd.json_normalize(df['attractions_features'].tolist())

# Display first few rows and info
print("DataFrame Info:")
print(df_expanded.info())
print("\nFirst few rows:")
display(df_expanded.head())

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               36 non-null     object
 1   attraction_type    36 non-null     object
 2   category           36 non-null     object
 3   max_height         36 non-null     object
 4   track_length       36 non-null     object
 5   max_speed          36 non-null     object
 6   g_force            36 non-null     object
 7   min_age            36 non-null     object
 8   min_height         36 non-null     object
 9   capacity_per_hour  36 non-null     int64 
dtypes: int64(1), object(9)
memory usage: 2.9+ KB
None

First few rows:


,name,attraction_type,category,max_height,track_length,max_speed,g_force,min_age,min_height,capacity_per_hour
0,Silver Star,Hyper Coaster,thrill,73 meters,1620 meters,127 km/h,4 G,8 years,140 cm,1750
1,Blue Fire Megacoaster,Launched Coaster,thrill,38 meters,1056 meters,100 km/h,3.8 G,7 years,130 cm,1720
2,Wodan – Timburcoaster,Wooden Coaster,thrill,40 meters,1050 meters,100 km/h,3.5 G,8 years,140 cm,1250
3,Voletarium,Flying Theater,family,16 meters,N/A,25 km/h,1.5 G,All ages,None,1400
4,Alpine Express Enzian,Powered Coaster,family,6 meters,430 meters,35 km/h,1.5 G,4 years,95 cm (with adult),900


Display first few rows and info

Save as parquet

In [20]:
df_expanded.to_parquet(parquet_file, index=False)
print(f"Data saved to {parquet_file}")

Data saved to ..\data\attraction_features.parquet


Verify the parquet file can be read

In [21]:
df_parquet = pd.read_parquet(parquet_file)
print("Verification - DataFrame Info:")
print(df_parquet.info())
print("\nVerification - First few rows:")
display(df_parquet.head())

Verification - DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               36 non-null     object
 1   attraction_type    36 non-null     object
 2   category           36 non-null     object
 3   max_height         36 non-null     object
 4   track_length       36 non-null     object
 5   max_speed          36 non-null     object
 6   g_force            36 non-null     object
 7   min_age            36 non-null     object
 8   min_height         36 non-null     object
 9   capacity_per_hour  36 non-null     int64 
dtypes: int64(1), object(9)
memory usage: 2.9+ KB
None

Verification - First few rows:


,name,attraction_type,category,max_height,track_length,max_speed,g_force,min_age,min_height,capacity_per_hour
0,Silver Star,Hyper Coaster,thrill,73 meters,1620 meters,127 km/h,4 G,8 years,140 cm,1750
1,Blue Fire Megacoaster,Launched Coaster,thrill,38 meters,1056 meters,100 km/h,3.8 G,7 years,130 cm,1720
2,Wodan – Timburcoaster,Wooden Coaster,thrill,40 meters,1050 meters,100 km/h,3.5 G,8 years,140 cm,1250
3,Voletarium,Flying Theater,family,16 meters,N/A,25 km/h,1.5 G,All ages,None,1400
4,Alpine Express Enzian,Powered Coaster,family,6 meters,430 meters,35 km/h,1.5 G,4 years,95 cm (with adult),900


In [30]:
from tqdm import tqdm

# Read the merged_rides parquet file
merged_rides = pd.read_parquet(data_dir / 'raw/51/merged_rides.parquet')

# Create a mapping dictionary from attraction name to its features
attraction_features_dict = df_expanded.set_index('name').to_dict('index')

# Function to get features for an attraction
def get_attraction_features(row):
    features = attraction_features_dict.get(row['ride_name'], {})
    for key, value in features.items():
        row[f'feature_{key}'] = value
    return row

# Apply the features to each row with tqdm progress bar
tqdm.pandas(desc="Adding features to rides")
merged_rides_with_features = merged_rides.progress_apply(get_attraction_features, axis=1)

# Save the updated DataFrame
merged_rides_with_features.to_parquet(data_dir / 'merged_rides_with_features.parquet', index=False)

# Verify the new file
print("\nVerification of merged rides with features:")
print(merged_rides_with_features.info())
print("\nSample of merged rides with features:")
display(merged_rides_with_features.head())

# Show which columns were added
new_columns = [col for col in merged_rides_with_features.columns if col not in merged_rides.columns]
print("\nNewly added feature columns:")
print(new_columns)

Adding features to rides:   0%|          | 13545/6268117 [00:49<5:36:30, 309.78it/s] Exception ignored in: <function tqdm.__del__ at 0x00000229BDE2AFC0>
Traceback (most recent call last):
  File "C:\Users\nevin\AppData\Roaming\Python\Python313\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\nevin\AppData\Roaming\Python\Python313\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Adding features to rides:   2%|▏         | 150773/6268117 [1:57:10<79:14:27, 21.44it/s]    


KeyboardInterrupt: 